# Unit 1: Set Indoor Navigation Stack

<img src="img/summit_xl_instruction_manual_unit1_intro_v2.png"/>

<img src="img/robotnik_logo.png" width="300"/>

<img src="img/robotignite_logo_text.png" width="300"/>

The first thing you will need for a robot that patroles is the ability to move around without crashing into everything. For this you will learn:<br>
* How to move the robot
* How to set the navigation stack to be able to send him a location and be able to reach that destination.
* How to send it a sequence of waypoints and execute those movements.

## Move Summit XL

In this case you have the standard wheel model with skid-steering. In escence this make it a normal vehicle that has manouver much more to get to a location and orientation. In th eproject you will use the omnidirectional wheels model.

To move it you just have to publish in the topic **/summit_xl_control/cmd_vel**.

<p style="background:#EE9023;color:white;">Exercise U1-1</p>

Publish through the web shell in the mentioned topic and see if you can make SummitXL move forwards, backwards, left, right and in circles.

<p style="background:#EE9023;color:white;">END Exercise U1-1</p>

You should have done something like this:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rostopic pub /summit_xl_control/cmd_vel geometry_msgs/Twist [TAB][TAB]
</th>
</tr>
</table>

With this command you can then fill in the structure that appears after hitting TAB key two consecutive times, to publish the message.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rostopic info /summit_xl_control/cmd_vel
</th>
</tr>
</table>

Like this you would get the type of message to publish in that topic.

As you can see the type of the message is the classical **geometry_msgs/Twist**. It will move as any differential drive that you have used. That is you move it by only publishing in the **linear.x** and the **angular.z**.<br>
Of ocurse this is not the way you will use it. You normally need to move it manually to make tests, so the best way is to create a keyboard teleop system that can move the SummitXL through the keyboard or any other similar device.<br>
Here you have an example of launch file that could be used for this matter:

**start_teleop.launch**

In [ ]:
<launch>
  <!-- turtlebot_teleop_key already has its own built in velocity smoother -->
  <node pkg="turtlebot_teleop" type="turtlebot_teleop_key.py" name="turtlebot_teleop_keyboard"  output="screen">
    <param name="scale_linear" value="0.5" type="double"/>
    <param name="scale_angular" value="1.5" type="double"/>
    <remap from="turtlebot_teleop_keyboard/cmd_vel" to="/summit_xl_control/cmd_vel"/>
  </node>
</launch>

There two point to talk about here:<br>
* turtlebot_teleop_key.py: This is the program used to capture the keys pressed in the terminal and converts them to Twist commands that are then published into the topic **turtlebot_teleop_keyboard/cmd_vel**

* remap: This remap is to transfer all the information published in **turtlebot_teleop_keyboard/cmd_vel** in the topic that SummitXL controller is listening to **/summit_xl_control/cmd_vel**.

Of ocurse you could publish directly into **/summit_xl_control/cmd_vel**, but then you would have to create your own version of **turtlebot_teleop_key.py**. Thanks to the ROS infraestructure, this way you recicle code, and get work done faster.

<p style="background:#EE9023;color:white;">Exercise U1-2</p>

Create a package that you  will use along the rest of this course where you can have all your programs. Use the name **my_sumit_xl_tools** for future refference.<br>
Then create your own launch for keyboard teleoperation based on the example and play around with SummitXL. See how it moves and its limitations. This is crucial to know what you can do with your robot.

<p style="background:#EE9023;color:white;">END Exercise U1-2</p>

Remember that you can also use the **interactive_markers** to move Summit_XL around. To have them operational you just have to launch the node **marker_server** from the package **interactive_marker_twist_server**.
This node just publishes by default in the topic **twist_marker_server/cmd_vel** Twist commands through the **RVIZ** interactive marker element.
You have more informaion <a href="http://wiki.ros.org/interactive_marker_twist_server">here</a>.

Here you have an example on how it should be launched:

**start_interactivemarker.launch**

In [ ]:
<launch>
    <node pkg="interactive_marker_twist_server" type="marker_server" name="twist_marker_server">
        <remap from="twist_marker_server/cmd_vel" to="/cmd_vel" />
    </node>
</launch>

Note that there is a **remap** to the topic where the Sumit_XL listens to **Twists** commands. In this case **/cmd_vel**, which is the standard.<br>
You now only have to add into the RVIZ session, an **interactive_marker** element and select the topic **/twist_marker_server/update**.

<img src="img/summit_xl_manual_unit1_interactivemarker.png" width="300"/>

When you launch the **start_interactivemarker.launch** and added the corresponding element in RVIZ, you shoudl get something similar to this:

<img src="img/summit_xl_manual_rviz_interactivezoom.png"/>

## How to navigate without map

Here you will learn how to reproduce the system that you launched in the Demo to make Summit XL navigate only with the map generated localy by the detection of obstacles.

Here we will go over the files and only point out particular elements for Summit XL. If you want to know more details on how navigation is performed in ROS, please do consider to do **RobotIgnites Navigation in ROS Course** before continuing.

### Create the launch file for move base

This is the core of how to make Summit XL move around. The package <a href="http://wiki.ros.org/move_base">move_base</a> makes all the path planning for you.<br>
You will use **move_base** for all the navigation, but here it only has Summit XL sensors to detect obstacles.<br>
Here you have an example of how **move_base** should be launched.

**move_base_odom.launch**

In [ ]:
<?xml version="1.0"?>
<!-- NEW SUMIT XL NAVIGATION -->
<launch>
   
    <arg name="no_static_map" default="true"/>
    
    <arg name="base_global_planner" default="navfn/NavfnROS"/>
    <arg name="base_local_planner" default="dwa_local_planner/DWAPlannerROS"/>
    <!-- <arg name="base_local_planner" default="base_local_planner/TrajectoryPlannerROS"/> -->
    
    <node pkg="move_base" type="move_base" respawn="false" name="move_base" output="screen">
        
        <param name="base_global_planner" value="$(arg base_global_planner)"/>
        <param name="base_local_planner" value="$(arg base_local_planner)"/>  
        <rosparam file="$(find sumit_xl_tools)/config/planner.yaml" command="load"/>
        
        <!-- observation sources located in costmap_common.yaml -->
        <rosparam file="$(find sumit_xl_tools)/config/costmap_common.yaml" command="load" ns="global_costmap" />
        <rosparam file="$(find sumit_xl_tools)/config/costmap_common.yaml" command="load" ns="local_costmap" />
        
        <!-- local costmap, needs size -->
        <rosparam file="$(find sumit_xl_tools)/config/costmap_local.yaml" command="load" ns="local_costmap" />
        <param name="local_costmap/width" value="5.0"/>
        <param name="local_costmap/height" value="5.0"/>
        
        <!-- static global costmap, static map provides size -->
        <rosparam file="$(find sumit_xl_tools)/config/costmap_global_static.yaml" command="load" ns="global_costmap" unless="$(arg no_static_map)"/>
        
        <!-- global costmap with laser, for odom_navigation_demo -->
        <rosparam file="$(find sumit_xl_tools)/config/costmap_global_laser.yaml" command="load" ns="global_costmap" if="$(arg no_static_map)"/>
        <param name="global_costmap/width" value="100.0" if="$(arg no_static_map)"/>
        <param name="global_costmap/height" value="100.0" if="$(arg no_static_map)"/>
    </node>

</launch>

Here we have some elements to comment on:<br>
Basically **move_base** takes in configuration files of .yaml format that define everything. From which **planner** its going to use, to how the maps are used or sensors. All of them will be located in a folder, which in this case is caled **config** inside your **my_sumit_xl_tools** package. So lets comment on them:

**costmap_common.yaml**

In [ ]:
footprint: [[0.35, -0.3], [0.35, 0.3], [-0.35,0.3], [-0.35, -0.3]]
footprint_padding: 0.01

robot_base_frame: base_link
update_frequency: 4.0
publish_frequency: 3.0
transform_tolerance: 0.5

resolution: 0.05

obstacle_range: 5.5
raytrace_range: 6.0

#layer definitions
static:
    map_topic: /map
    subscribe_to_updates: true

obstacles_laser:
    observation_sources: hokuyo_laser
    hokuyo_laser: {sensor_frame: hokuyo_base_link, data_type: LaserScan, clearing: true, marking: true, topic: hokuyo_base/scan, inf_is_valid: true}

inflation:
    inflation_radius: 1.0


Comment on some elements that reffer to Summit XL directly:
*  **footprint and footprint_padding**: Parameters define the simplified model of the robot into a simple box that surrounds the robot. If you want the robot to navigate always very close to objects, make the padding **smaller**. In this case the value is **0.1** because this model of Summit with normal wheels tends to have certain error when turning, and therefore it not as manuverable as its brother Summit XL with Omnidirectional wheels.
* **obstacles_laser**: here the sensor from which the laser data for navigation is defined. In this case the topic is **/hokuyo_base/scan**.

Also comment on the fact that this file **costmap_common.yaml** is loaded for both namespaces **global_costmap** and **local_costmap**. This menas that it will be used in the local and global planning and generation of both costmaps. Is just a way to recicle code.

**costmap_local.yaml**

In [ ]:
global_frame: odom
rolling_window: true

plugins:
  - {name: obstacles_laser,           type: "costmap_2d::ObstacleLayer"}
  - {name: inflation,                 type: "costmap_2d::InflationLayer"}

Here comment:
* The **plugins**: here the plugins for the obstacle detection and inflation are loaded and executed. 
* **global_frame**: In case of the LocalPlanning, it normally can be set always to odom, but it could also be set to /map... Depending on your preferences.

Also we have to comment on the parameters that are set outside the **yaml** file. This is normaly done when these parameters are needed to be easy to access. As you can see they are set in the **local_costmap** namse space:<br>
* **local_costmap/width**
* **local_costmap/height**

Now in the next loading of the **costmap_global_XX.yaml** , see that there are two options, that will be loaded one or the other, depending on the value of the boolean argument **no_static_map**. In case of having **NO map** you will load the second one **costmap_global_laser.yaml** by setting **no_static_map** to **true**. If **false**, then it will load the **costmap_global_static.yaml** file, which is for situations when you **have** a map.<br>
We will talk about both files in this section to have a better understanding.

**costmap_global_laser.yaml**

In [ ]:
global_frame: odom
rolling_window: true
track_unknown_space: true

plugins:
  - {name: obstacles_laser,           type: "costmap_2d::ObstacleLayer"}
  - {name: inflation,                 type: "costmap_2d::InflationLayer"}

**costmap_global_static.yaml**

In [ ]:
global_frame: map
rolling_window: false
track_unknown_space: true

plugins:
  - {name: static,                  type: "costmap_2d::StaticLayer"}
  - {name: inflation,               type: "costmap_2d::InflationLayer"}

So as you can see the only differences between moving with a map or without are:
* **global_frame**: This is set to MAP when you have one, to **odom** if you dont have a map.
* **plugins**: The plugins are different becuase they will be based on an existing map or not, because its completely different the way to obtain the costmaps. One is with map data and the other one with laser data.

And finally coment on the **planners selected**:

In [ ]:
    <arg name="base_global_planner" default="navfn/NavfnROS"/>
    <arg name="base_local_planner" default="dwa_local_planner/DWAPlannerROS"/>
    <!-- <arg name="base_local_planner" default="base_local_planner/TrajectoryPlannerROS"/> -->

**planner.yaml**

In [ ]:
controller_frequency: 5.0
recovery_behaviour_enabled: true

NavfnROS:
  allow_unknown: true # Specifies whether or not to allow navfn to create plans that traverse unknown space.
  default_tolerance: 0.1 # A tolerance on the goal point for the planner.

TrajectoryPlannerROS:
  # Robot Configuration Parameters
  acc_lim_x: 2.5
  acc_lim_theta:  3.2

  max_vel_x: 1.0
  min_vel_x: 0.0

  max_vel_theta: 1.0
  min_vel_theta: -1.0
  min_in_place_vel_theta: 0.2

  holonomic_robot: false
  escape_vel: -0.1

  # Goal Tolerance Parameters
  yaw_goal_tolerance: 0.1
  xy_goal_tolerance: 0.2
  latch_xy_goal_tolerance: false

  # Forward Simulation Parameters
  sim_time: 2.0
  sim_granularity: 0.02
  angular_sim_granularity: 0.02
  vx_samples: 6
  vtheta_samples: 20
  controller_frequency: 20.0

  # Trajectory scoring parameters
  meter_scoring: true # Whether the gdist_scale and pdist_scale parameters should assume that goal_distance and path_distance are expressed in units of meters or cells. Cells are assumed by default (false).
  occdist_scale:  0.1 #The weighting for how much the controller should attempt to avoid obstacles. default 0.01
  pdist_scale: 0.75  #     The weighting for how much the controller should stay close to the path it was given . default 0.6
  gdist_scale: 1.0 #     The weighting for how much the controller should attempt to reach its local goal, also controls speed  default 0.8

  heading_lookahead: 0.325  #How far to look ahead in meters when scoring different in-place-rotation trajectories
  heading_scoring: false  #Whether to score based on the robot's heading to the path or its distance from the path. default false
  heading_scoring_timestep: 0.8   #How far to look ahead in time in seconds along the simulated trajectory when using heading scoring (double, default: 0.8)
  dwa: true #Whether to use the Dynamic Window Approach (DWA)_ or whether to use Trajectory Rollout
  simple_attractor: false
  publish_cost_grid_pc: true  

  # Oscillation Prevention Parameters
  oscillation_reset_dist: 0.25 #How far the robot must travel in meters before oscillation flags are reset (double, default: 0.05)
  escape_reset_dist: 0.1
  escape_reset_theta: 0.1

DWAPlannerROS:
  # Robot configuration parameters  
  acc_lim_x: 2.5
  acc_lim_y: 0
  acc_lim_th: 3.2

  max_vel_x: 0.5
  min_vel_x: 0.0
  max_vel_y: 0
  min_vel_y: 0

  max_trans_vel: 0.5
  min_trans_vel: 0.1
  max_rot_vel: 1.0
  min_rot_vel: 0.2

  # Goal Tolerance Parameters
  yaw_goal_tolerance: 0.1
  xy_goal_tolerance: 0.2
  latch_xy_goal_tolerance: false

  # # Forward Simulation Parameters
  # sim_time: 2.0
  # sim_granularity: 0.02
  # vx_samples: 6
  # vy_samples: 0
  # vtheta_samples: 20
  # penalize_negative_x: true

  # # Trajectory scoring parameters
  # path_distance_bias: 32.0 # The weighting for how much the controller should stay close to the path it was given
  # goal_distance_bias: 24.0 # The weighting for how much the controller should attempt to reach its local goal, also controls speed
  # occdist_scale: 0.01 # The weighting for how much the controller should attempt to avoid obstacles
  # forward_point_distance: 0.325 # The distance from the center point of the robot to place an additional scoring point, in meters
  # stop_time_buffer: 0.2  # The amount of time that the robot must stThe absolute value of the veolicty at which to start scaling the robot's footprint, in m/sop before a collision in order for a trajectory to be considered valid in seconds
  # scaling_speed: 0.25 # The absolute value of the veolicty at which to start scaling the robot's footprint, in m/s
  # max_scaling_factor: 0.2 # The maximum factor to scale the robot's footprint by

  # # Oscillation Prevention Parameters
  # oscillation_reset_dist: 0.25 #How far the robot must travel in meters before oscillation flags are reset (double, default: 0.05)

Here we could comment loads of parameters. Just point out a few really important:
* yaw_goal_tolerance: This will indicate how precise you want the robot to be in relation to the given pose. In this case the orientation in the XY plane.
*  xy_goal_tolerance: The same but refferd to the position in the plane XY.
* holonomic_robot: This is important because the Summit XL when it has Omnidirectional wheels, it can be considered Holonomic. In the cirrent SUmmit Model you are using, its not.

<a href="http://wiki.ros.org/dwa_local_planner">DWAPlannerROS</a> is the planner used for the local planner. You could also use the <a href="http://wiki.ros.org/base_local_planner">TrajectoryPlannerROS</a>. The main difference is the way the velocities are sampled. But they perform in general cases more or less at the same level.

<p style="background:#EE9023;color:white;">Exercise U1-3</p>

* Inside the new **my_sumit_xl_tools** create the files given examples and the launch file **move_base_odom.launch**.
* Then launch rviz through the command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rosrun rviz rviz
</th>
</tr>
</table>

* Add the RVIZ elements: 

<img src="img/summit_xl_manual_unit1_rvizelements_odomnav.png"/>

* The fixed frame: has to be **odom**, you dont have any map frame for the moment.
* Note the two paths, one for the local and another for the global planner.
* Two maps. One for the **LocalCostmap** and the other for the **GlobalCostmap**.
* **move_base_simple/goal** will be the value of the topic for the **Pose**
* RobotFootprint: you can select the local or global.Its usefull to see if you set the correct robot dimensions.

Remember to save the configuration of RVIZ when finished to use it afterwards. Name it **navigation_odom** and save it in a folder called **rviz**.

<p style="background:#EE9023;color:white;">END Exercise U1-3</p>

You should get the same result as in the Demo:

<img src="img/summit_xl_instruction_manual_unit1_navodom1.png"/>

Check that there is a difference in the Global and Local Costmaps, the size. Try to change it.

<img src="img/summit_xl_instruction_manual_unit1_navodom2.png"/>

<img src="img/summit_xl_instruction_manual_unit1_navodom3.png"/>

## Navigate with Map

Now you are going to learn how to navigate with a map. But first you have to generate that map:

### Create a Map

To create a Map you need to launch two elements: **GMapping** and **MoveBase**.
* **GMapping**: in the one in charge of generating and saving the map. More info <a href="http://wiki.ros.org/gmapping">here</a>.
* **MoveBase**: The one in charge of sending the commands to /cmd_vel to move the robot around based on sensors and now a map.

So you can create a launch file that launches both:

**start_mapping.launch**

In [ ]:
<launch>

  <!--- Run gmapping -->
  <include file="$(find my_sumit_xl_tools)/launch/gmapping.launch" />

  <!--- Run Move Base -->
  <include file="$(find my_sumit_xl_tools)/launch/move_base_map.launch" />

</launch>

So first, take a look at the **gmapping.launch**. It should be something similar to this:

In [ ]:
<launch>
    <node pkg="gmapping" type="slam_gmapping" name="slam_gmapping" output="screen">
      <!-- simulation remap from="scan" to="/hokuyo_laser_topic"/ -->
      <!-- real -->
      <!-- remap from="scan" to="/scan_filtered"/ -->
      <remap from="scan" to ="/hokuyo_base/scan"/>
      <param name="base_frame" value="base_footprint"/>
      <param name="odom_frame" value="odom"/>
      <!-- Process 1 out of every this many scans (set it to a higher number to skip more scans)  -->
    <param name="throttle_scans" value="1"/>

    <param name="map_update_interval" value="5.0"/> <!-- default: 5.0 -->

    <!-- The maximum usable range of the laser. A beam is cropped to this value.  -->
    <param name="maxUrange" value="5.0"/>

    <!-- The maximum range of the sensor. If regions with no obstacles within the range of the sensor should appear as free space in the map, set maxUrange < maximum range of the real sensor <= maxRange -->
    <param name="maxRange" value="10.0"/>
      <param name="sigma" value="0.05"/>
      <param name="kernelSize" value="1"/>
      <param name="lstep" value="0.05"/>
      <param name="astep" value="0.05"/>
      <param name="iterations" value="5"/>
      <param name="lsigma" value="0.075"/>
      <param name="ogain" value="3.0"/>
      <param name="lskip" value="0"/>
      <param name="srr" value="0.1"/>
      <param name="srt" value="0.2"/>
      <param name="str" value="0.1"/>
      <param name="stt" value="0.2"/>
      <param name="linearUpdate" value="0.2"/>
      <param name="angularUpdate" value="0.1"/>
      <param name="temporalUpdate" value="3.0"/>
      <param name="resampleThreshold" value="0.5"/>
      <param name="particles" value="100"/>
      <param name="xmin" value="-50.0"/>
      <param name="ymin" value="-50.0"/>
      <param name="xmax" value="50.0"/>
      <param name="ymax" value="50.0"/>
      <param name="delta" value="0.05"/>
      <param name="llsamplerange" value="0.01"/>
      <param name="llsamplestep" value="0.01"/>
      <param name="lasamplerange" value="0.005"/>
      <param name="lasamplestep" value="0.005"/>
    </node>
</launch>

The most import ant values are:
* maxUrange and maxrange: This is how far your laser will be considered to create the map. Greater range will create maps faster and its less probable that the robot gets lost. The downside its that consumes more resources.
* throttle_scans: Very usefull to lower resource consumption.

If you need mor information about all these parameters, please go to the slam_gmapping node docs: http://docs.ros.org/hydro/api/gmapping/html/

**move_base_map.launch**

In [ ]:
<?xml version="1.0"?>
<!-- NEW SUMIT XL NAVIGATION -->
<launch>
   
    <arg name="no_static_map" default="false"/>
    
    <arg name="base_global_planner" default="navfn/NavfnROS"/>
    <arg name="base_local_planner" default="dwa_local_planner/DWAPlannerROS"/>
    <!-- <arg name="base_local_planner" default="base_local_planner/TrajectoryPlannerROS"/> -->
    
    <node pkg="move_base" type="move_base" respawn="false" name="move_base" output="screen">
        
        <param name="base_global_planner" value="$(arg base_global_planner)"/>
        <param name="base_local_planner" value="$(arg base_local_planner)"/>  
        <rosparam file="$(find sumit_xl_tools)/config/planner.yaml" command="load"/>
        
        <!-- observation sources located in costmap_common.yaml -->
        <rosparam file="$(find sumit_xl_tools)/config/costmap_common.yaml" command="load" ns="global_costmap" />
        <rosparam file="$(find sumit_xl_tools)/config/costmap_common.yaml" command="load" ns="local_costmap" />
        
        <!-- local costmap, needs size -->
        <rosparam file="$(find sumit_xl_tools)/config/costmap_local.yaml" command="load" ns="local_costmap" />
        <param name="local_costmap/width" value="5.0"/>
        <param name="local_costmap/height" value="5.0"/>
        
        <!-- static global costmap, static map provides size -->
        <rosparam file="$(find sumit_xl_tools)/config/costmap_global_static.yaml" command="load" ns="global_costmap" unless="$(arg no_static_map)"/>
        
        <!-- global costmap with laser, for odom_navigation_demo -->
        <rosparam file="$(find sumit_xl_tools)/config/costmap_global_laser.yaml" command="load" ns="global_costmap" if="$(arg no_static_map)"/>
        <param name="global_costmap/width" value="100.0" if="$(arg no_static_map)"/>
        <param name="global_costmap/height" value="100.0" if="$(arg no_static_map)"/>
    </node>

</launch>

As you can see its exactly the same as the move base without map, just that we deactivate the **no_static_map** argument. This will change the **global_costmap/global_frame** to **map** and launch the **costmap_2d::StaticLayer** plugin.

<p style="background:#EE9023;color:white;">Exercise U1-4</p>

Now you just have to launch the **start_mapping.launch** to start all the system and start creating the map. You also need to launch rviz adding the propper elements. Once you finish mapping all the area, continue to the next secction **without closing anything**.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
roslaunch my_sumit_xl_tools start_mapping.launch
</th>
</tr>
</table>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p><br>
rosrun rviz rviz
</th>
</tr>
</table>

* Add the RVIZ elements: 

<img src="img/summit_xl_manual_unit1_rvizelements_mapcreate.png"/>

* The fixed frame: has to be **map**.
* Note the two paths, one for the local and another for the global planner.
* Two maps. One for the **LocalCostmap** and the other for the **GlobalCostmap**. Its really only important to have the localCostmap to see the inmediate dangers.
* Add an extra map for the global map to be able to see the generation of the map.
* **move_base_simple/goal** will be the value of the topic for the **Pose**
* RobotFootprint: you can select the local or global.Its usefull to see if you set the correct robot dimensions.

Save the RVIZ configuration in a file called **create_map.rviz**.

You need to now move the robot around to generate a complete map. You can use the **start_teleop.launch** you created or **send poses 2D** in RVIZ. Use the keyboard in dificult to navigate places, and the poses to easy big areas.

Now start moving the robot around the room to create the map.

<p style="background:#EE9023;color:white;">Exercise U1-4</p>

You should get something similar to this:

<img src="img/summit_xl_instruction_manual_unit1_createmap1.png" width="800"></img>

<img src="img/summit_create_map.gif"></img>

### Save the created map

Once you have all the map as you would like, then you have to save it to be able to use it for navigation.<br>
For this you have to execute the following command where you want the map files to be saved. You have a sugesstion on where to put it but its up to you:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
roscd my_sumit_xl_tools<br>
mkdir maps<br>
cd maps<br>
rosrun map_server map_saver -f mymap
</th>
</tr>
</table>

This should have generated two files: **mymap.pgm and mymap.yaml**.<br>
You will use the **pgm** image for cleaning the map generated, and the **yaml** is the file directly used by the PathPLanning afterwards. It also contains valuable information like the path to the image of the maps dimmensions.

<img src="img/fetch_map.png" width="600"></img>

### Navigate with Map that you just created

**start_navigation_with_map.launch**

In [ ]:
<launch>

    <!-- Run the map server -->
    <arg name="map_file" default="$(find my_sumit_xl_tools)/maps/mymap.yaml"/>
    <node name="map_server" pkg="map_server" type="map_server" args="$(arg map_file)" />
    
    <!--- Run AMCL -->
    <include file="$(find my_sumit_xl_tools)/launch/amcl.launch" />
    
    <!--- Run Move Base -->
    <include file="$(find my_sumit_xl_tools)/launch/move_base_map.launch" />

</launch>

Here there three main parts:
* **map_server**: The Node that publishes the map saved.
* **amcl**: The node that localises Summit XL based on map readings
* **move_base**: This move base has the configuration of moving with a map.

So really there is only one launch file to comment on which is:

**amcl.launch**

In [ ]:
<launch>
<node pkg="amcl" type="amcl" name="amcl" output="screen">

  <remap from="scan" to="/hokuyo_base/scan" />
  <remap from="cmd_vel" to="/summit_xl_controller/cmd_vel"/>

  <!-- Publish scans from best pose at a max of 10 Hz -->
  <param name="odom_model_type" value="diff"/>
  <param name="odom_alpha5" value="0.1"/>
  <param name="transform_tolerance" value="0.2" />
  <param name="gui_publish_rate" value="10.0"/>
  <param name="laser_max_beams" value="30"/>
  <param name="min_particles" value="500"/>
  <param name="max_particles" value="5000"/>
  <param name="kld_err" value="0.05"/>
  <param name="kld_z" value="0.99"/>
  <param name="odom_alpha1" value="0.2"/>
  <param name="odom_alpha2" value="0.2"/>
  <param name="odom_alpha3" value="0.8"/>
  <param name="odom_alpha4" value="0.2"/>
  <param name="laser_z_hit" value="0.5"/>
  <param name="laser_z_short" value="0.05"/>
  <param name="laser_z_max" value="0.05"/>
  <param name="laser_z_rand" value="0.5"/>
  <param name="laser_sigma_hit" value="0.2"/>
  <param name="laser_lambda_short" value="0.1"/>
  <param name="laser_lambda_short" value="0.1"/>
  <param name="laser_model_type" value="likelihood_field"/>
  <param name="laser_likelihood_max_dist" value="2.0"/>
  <param name="update_min_d" value="0.2"/>
  <param name="update_min_a" value="0.5"/>
  <param name="odom_frame_id" value="odom"/>
  <param name="base_frame_id" value="base_footprint"/>
  <param name="global_frame_id" value="map"/>
  <param name="resample_interval" value="1"/>
  <param name="transform_tolerance" value="0.1"/>
  <param name="recovery_alpha_slow" value="0.0"/>
  <param name="recovery_alpha_fast" value="0.0"/>
  <param name="initial_pose_x" value ="0.0"/>
  <param name="initial_pose_y" value ="0.0"/>
  <param name="initial_pose_a" value ="0.0"/>

</node>

</launch>

<p style="background:#EE9023;color:white;">Exercise U1-5</p>

Launch the node and check in RViz that the Summit robot correctly localizes itseld in the Map.

* Add the RVIZ elements: 

<img src="img/summit_xl_manual_unit1_rvizelements_navmap.png"></img>

* The only real change from the **create_map.rviz** file is the **PoseArray** element susbrcibed to the **/particlecloud** topic.This will give the estimated position whereSummit thinks it is.

<p style="background:#EE9023;color:white;">END Exercise U1-5</p>

If you've done all the previous steps correctly, you should see in Rviz something like this:

<img src="img/summit_xl_instruction_manual_unit1_navigationmap_final.png" width="600"></img>

See how when you move , the estimated locations seem to concentrate on the real place. This is a test on how well your loacalization is working.

<img src="img/summit_navigation_map.gif"></img>

### Create a Table of Positions and send Summit there

Once you've checked that localization is working fine, you'll need to create a table with 3 different spots in the environemnt. For each point, assign a tag (with the name of the spot) alongside its coordinates in the Map.

The 3 spots you'll have to register into the table are the next ones:

* Room1 Center
<br>
<img src="img/fetch_room.png" width="500"></img>
<br>
* Facing the Turtle's Poster
<br>
<img src="img/turtle_poster.png" width="500"></img>
* Besides the Table
<br>
<img src="img/besides_table.png" width="500"></img>
<br>

You can access the coordinates of each position by looking into the topics the amcl node publishes on. The only data you really need to save are the position and orientation. 

Next you have a screenshot of the <i>/amcl_pose</i> topic:

<img src="img/echo_amcl_pose.png" width="800"></img>

Create a file named <b>spots.txt</b> and write in the pose data you've get from the 3 spots.

### Create a ROS Service that saves the different spots into a file

In the previous section, you created a file with the coordinates of the 3 spots. But the whole process was too cumbersome, don't you think so? Why don't we create a ROS Service that does the work for us? Sounds like a good idea, right? Then let's do it!

So now, you'll create a ROS program that will do the following:

1. It will launch a node named <b>spot_recorder</b>.
2. This node will contain a Service Server named <b>/save_spot</b>, that will take an string as input.
3. When this service is called, it will store the current coordinates of the robot (position and orientation values) alongside with a label, that will be the string provided on the service call.
4. When the <i>end</i> string is provided in the service call, the node will write all the values stored into a file named <b>spots.txt</b>.
5. Finally, the service will return a message indicating if the file has been saved correctly.

In order to achieve this, let's divide it into smaller parts.

#### 1 Service Message

First of all, you'll have to determine the kind of data you need for your service message.

1. Determine what input data you need ( <span class="ign_blue"><b>request</b></span> )
2. Determine what data you want the service to return ( <span class="ign_blue"><b>response</b></span> )

Next, search if there exists an already built message in the system that suits your needs. If it doesn't, then you'll have to create your own custom message with the data you want.

If this is the case, then do the following. Inside the pacakge you just created, create a new directory named <b>srv</b>. Inside this directory, create a file named <b>SumitXLPose.srv</b>, that will contain the definition of your service message. 

This file could be something like this:

Now you'll have to modify the <b>package.xml</b> and <b>CMakeLists.txt</b> files of your package, in order to compile the new message. Assuming the only dependency you added to your package is <b>rospy</b>, you should do the following modifications to these files:

#### In CMakeLists.txt

You will have to edit/uncomment four functions inside this file:

* find_package()
* add_service_files()
* generate_messages()
* catkin_package()

add_service_files(
  FILES
  MyCustomServiceMessage.srv
)

generate_messages(
  DEPENDENCIES
  std_msgs
)

#### In package.xml:

Add this 2 lines to the file:

When you've finshed with the modifications, compile your package and check that your message has been correctly compiled and is ready to use. In order to check this, you can use the command <b><i>rossrv show MyServiceMessage</i></b>.

catkin_make output:

<img src="img/catkin_make_output.png" width="600"></img>

rossrv show output:

<img src="img/service_msg_output.png" width="600"></img>

#### Service Code

Once your message is created, you're ready to use it in your service! So, let's write the code for our service. Inside the src directory of your package, create a file named <b>spots_to_file.py</b>. Inside this file, write the code necessary for you Service.

In [ ]:
#! /usr/bin/env python
import os
import rospy
from sumit_xl_tools.srv import SumitXLPose, SumitXLPoseResponse, SumitXLPoseRequest
from geometry_msgs.msg import PoseWithCovarianceStamped
import rospkg

"""
# topic: /amcl_pose
# geometry_msgs/PoseWithCovarianceStamped 

std_msgs/Header header                                                                                                                                       
  uint32 seq                                                                                                                                                 
  time stamp                                                                                                                                                 
  string frame_id                                                                                                                                            
geometry_msgs/PoseWithCovariance pose                                                                                                                        
  geometry_msgs/Pose pose                                                                                                                                    
    geometry_msgs/Point position                                                                                                                             
      float64 x                                                                                                                                              
      float64 y                                                                                                                                              
      float64 z                                                                                                                                              
    geometry_msgs/Quaternion orientation                                                                                                                     
      float64 x                                                                                                                                              
      float64 y                                                                                                                                              
      float64 z                                                                                                                                              
      float64 w                                                                                                                                              
  float64[36] covariance
"""

class SummitXLPose(object):
    def __init__(self):
        self.pose_service = rospy.Service('/get_pose_sumitxl_service', SumitXLPose , self.pose_service_callback) # create the Service called my_service with the defined callback
        rospy.Subscriber("/amcl_pose", PoseWithCovarianceStamped, self.amcl_pose_callback)
        self.pose_now = PoseWithCovarianceStamped()
        
        rospack = rospkg.RosPack()
        self.spot_file_path = rospack.get_path('my_sumit_xl_tools') + "/spots/spots_saved.txt"
        
        try:
            os.remove(self.spot_file_path)
        except:
            rospy.loginfo("File Not Found "+str(self.spot_file_path) )
        # Init File for Spot saving
        file2write=open(self.spot_file_path,'a')
        file2write.write("### This is the Spot Saving File ###\n")
        file2write.close()
        
    def start_loop_service(self):
        rospy.spin() # mantain the service open.
        
    def pose_service_callback(self, request):
        spot_name = request.label
        
        with open(self.spot_file_path, "a") as myfile:
            spot_pose = self.pose_now.pose.pose
            myfile.write(spot_name+":\n"+str(spot_pose)+"\n\n")

        response = SumitXLPoseResponse()
        response.navigation_successfull = True
        response.message = "Spot <"+spot_name+"> Saved"

        return response
    
    def amcl_pose_callback(self, message):
        self.pose_now = message
        


def startPoseService():
    rospy.init_node('sumitxl_poseservice_server')
    sumitpose = SummitXLPose()
    sumitpose.start_loop_service()

if __name__ == "__main__":
    startPoseService()
    

Create a launch file for the node you've just created. You can also launch this node in the same launch file you've created to launch the slam_gmapping node. It's up to you.

Using the keyboard teleop, move the robot to the 3 different spots showed above. At each one of these spots, perform a service call to the service you've just created. In the service call, provide the string with the name you want to give to each spot.
<br><br>
For example, when you are at the center of a room1, call your service like this:

Repeat this for the 3 different spots. When finished, do a last service call providing <i>end</i> as the string in order to create the file.

### Create a ROS program that interacts with the Navigation Stack

In the Step 2 of this project you created a table with 3 different spots, each one associated with a label. You may have been wondering why you were asked to do that, right? Well, then now you'll get the answer!
<br><br>
In this final Step you will have to create a ROS node that will interact with the Navigation Stack. This node will contain a Service that will take a string as input. This string will indicate one of the labels you selected in Step 2 for the different spots in the simulation. When this service receives a call with the label, it will get the coordinates (position and orientation) associated to that label, and will send this coordinates to the move_base Action Server. If it finished correctly, the service will return an "OK" message.

Summarizing, you will have to create a node which contains:

* A Service that will take an String as input. Those strings will be the labels you selected in Step 2 for each one of the spots.
* An Action Client that will send goals to the move_base Action Server.

In order to achieve this, follow the next steps:<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">1. Create the package</p>
</th>
</tr>
</table>

Create the package called <i>my_summit_navigation</i> that will contain all the files related to this project. Remember to specify the <b>rospy</b> dependency.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">2. Create the service message</p>
</th>
</tr>
</table>

First of all, you'll have to determine the kind of data you need for your mesage.

1. Determine what input data you need ( <span class="ign_blue"><b>request</b></span> )
2. Determine what data you want the service to return ( <span class="ign_blue"><b>response</b></span> )

Next, search if there exists an already built message in the system that suits your needs. If it doesn't, then you'll have to create your own custom message with the data you want.

If this is the case, then do the following. Inside the pacakge you just created, create a new directory named <b>srv</b>. Inside this directory, create a file named <b>MyServiceMessage.srv</b>, that will contain the definition of your service message. 

This file could be something like this:

Now you'll have to modify the <b>package.xml</b> and <b>CMakeLists.txt</b> files of your package, in order to compile the new message. Assuming the only dependency you added to your package is <b>rospy</b>, you should do the following modifications to these files:

#### In CMakeLists.txt

You will have to edit/uncomment four functions inside this file:

* find_package()
* add_service_files()
* generate_messages()
* catkin_package()

add_service_files(
  FILES
  MyCustomServiceMessage.srv
)

generate_messages(
  DEPENDENCIES
  std_msgs
)

#### In package.xml:

Add this 2 lines to the file:

When you've finshed with the modifications, compile your package and check that your message has been correctly compiled and is ready to use. In order to check this, you can use the command <b><i>rossrv show MyServiceMessage</i></b>.

catkin_make output:

<img src="img/catkin_make_output.png" width="600"></img>

rossrv show output:

<img src="img/service_msg_output.png" width="600"></img>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">3. Create the Service.</p>
</th>
</tr>
</table>

Once your message is created, you're ready to use it in your service! So, let's write the code for our service. Inside the src directory of your package, create a file named <b>my_navigation_program.py</b>. Inside this file, write the code necessary for you Service.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">4. Create the Action Client</p>
</th>
</tr>
</table>

Inside the src directory, create a file that will contain the code of an Action Client.

Once you've finished writting your code, it's time to test it! Call your service providing one of the labels, and check that the Summit robot navigates to that spot. For example, if your label is "fetch_room", type in a web shell the following:

## Add WayPoints

So there you have it. Now your summit XL can navigate around one position at a time. But what if you want to generate in an easy way a sequence of waypoints so that it patroles around the **main interest points of the area**?

You are going to use a ROS package called **follow_waypoints**, you can get more information http://wiki.ros.org/follow_waypoints.
This package basically tracks the **Estimate pose** that you place in RVIZ and stores it. Then when you have finished you just have to publish in a topic that starts sending those positions to the movebase system.

### Steps to do it:

The first step is to start the start_navigation_with_map.launch if you don't have it running:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
roslaunch my_sumit_xl_tools start_navigation_with_map.launch
</th>
</tr>
</table>

The Second step is to start the waypoint server. This server will listen to publications into the topic **/initialpose** and store those poses until its instructed to send them to move_base to be executed:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
roslaunch follow_waypoints follow_waypoints.launch
</th>
</tr>
</table>

Then you have to start RVIZ, and add a **PoseArray** element, susbcribed to the topic **/waypoints**, and change its name to WayPoints. This will now show all the waypoints you set from now on.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p><br>
rosrun rviz rviz
</th>
</tr>
</table>

<img src="img/summit_xl_instruction_manual_unit1_waypoints1.png" width="600"></img>

You can change the colour also if you want. In this example its changed to black to see it better.

Now you have to set the waypoints, for this you have to select the **PoseEstimate** and set it where you want.

<img src="img/summit_xl_instruction_manual_unit1_waypoints3.png" width="100"></img>

You should get something similar to this:

<img src="img/summit_xl_instruction_manual_unit1_waypoints2.png" width="600"></img>

You should also get in webshell where you launched the waypoint server a message stating that it recieved the waypoint:

In [ ]:
[INFO][139816504842048][/follow_waypoints/execute:92]: Waiting to recieve waypoints via Pose msg on topic /initialpose         
[INFO][139816504842048][/follow_waypoints/execute:93]: To start following waypoints: 'rostopic pub /path_ready std_msgs/Empty -
1'                                                                                                                             
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint

Now you just have to publish in the topic **/path_ready** to start sending waypoints to the movebase.

If you want to see the global path generated just activate it in RVIZ.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #3</p><br>
rostopic pub /path_ready std_msgs/Empty -1
</th>
</tr>
</table>

You should get something similar to this for each waypoint executed:

<img src="img/summit_xl_instruction_manual_unit1_waypoints4.png" width="600"></img>

<img src="img/summit_xl_instruction_manual_unit1_waypoints5.png" width="600"></img>

As you can see, its a nice way to do it , but its a bit cumbersome and messy. Specially because it has to change the initial position of the robot which is not very desirable. This is because this waypoint manager listenes to the topic **/initialpose**. Th eproblem is that **amlc localiser** also. Thats why when changed, the robot localises itself also in another place.<br>
So the next step is creating your own version. So you will part from the code that you just used that is installed in the system and is courtsesy of <a href="https://github.com/danielsnider">Daniel Snider</a> ,who created the original Waypoints manager <a href="https://github.com/danielsnider/follow_waypoints">Program</a>.

In order to avoid touching the **/initialpose**, the topic that the **follow_points** listens to, this has to be changed. Not only that put it has to be configurable. See here the final code:

**custom_follow_waypoints.py**

In [ ]:
#!/usr/bin/env python

import threading
import rospy
import actionlib

from smach import State,StateMachine
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from geometry_msgs.msg import PoseWithCovarianceStamped, PoseArray
from std_msgs.msg import Empty

waypoints = []

class FollowPath(State):
    def __init__(self):
        State.__init__(self, outcomes=['success'], input_keys=['waypoints'])
        self.frame_id = rospy.get_param('~goal_frame_id','map')
        # Get a move_base action client
        self.client = actionlib.SimpleActionClient('move_base', MoveBaseAction)
        rospy.loginfo('Connecting to move_base...')
        self.client.wait_for_server()
        rospy.loginfo('Connected to move_base.')

    def execute(self, userdata):
        global waypoints
        # Execute waypoints each in sequence
        for waypoint in waypoints:
            # Break if preempted
            if waypoints == []:
                rospy.loginfo('The waypoint queue has been reset.')
                break
            # Otherwise publish next waypoint as goal
            goal = MoveBaseGoal()
            goal.target_pose.header.frame_id = self.frame_id
            goal.target_pose.pose.position = waypoint.pose.pose.position
            goal.target_pose.pose.orientation = waypoint.pose.pose.orientation
            rospy.loginfo('Executing move_base goal to position (x,y): %s, %s' %
                    (waypoint.pose.pose.position.x, waypoint.pose.pose.position.y))
            rospy.loginfo("To cancel the goal: 'rostopic pub -1 /move_base/cancel actionlib_msgs/GoalID -- {}'")
            self.client.send_goal(goal)
            self.client.wait_for_result()
        return 'success'

def convert_PoseWithCovArray_to_PoseArray(waypoints):
    """Used to publish waypoints as pose array so that you can see them in rviz, etc."""
    poses = PoseArray()
    poses.header.frame_id = 'map'
    poses.poses = [pose.pose.pose for pose in waypoints]
    return poses

class GetPath(State):
    def __init__(self):
        State.__init__(self, outcomes=['success'], input_keys=['waypoints'], output_keys=['waypoints'])
        # Create publsher to publish waypoints as pose array so that you can see them in rviz, etc.
        self.poseArray_publisher = rospy.Publisher('/waypoints', PoseArray, queue_size=1)
        self._custom_waypointstopic = rospy.get_param('~custom_waypointstopic','/my_waypoints_list')
        # Start thread to listen for reset messages to clear the waypoint queue
        def wait_for_path_reset():
            """thread worker function"""
            global waypoints
            while not rospy.is_shutdown():
                data = rospy.wait_for_message('/path_reset', Empty)
                rospy.loginfo('Recieved path RESET message')
                self.initialize_path_queue()
                rospy.sleep(3) # Wait 3 seconds because `rostopic echo` latches
                               # for three seconds and wait_for_message() in a
                               # loop will see it again.
        reset_thread = threading.Thread(target=wait_for_path_reset)
        reset_thread.start()

    def initialize_path_queue(self):
        global waypoints
        waypoints = [] # the waypoint queue
        # publish empty waypoint queue as pose array so that you can see them the change in rviz, etc.
        self.poseArray_publisher.publish(convert_PoseWithCovArray_to_PoseArray(waypoints))

    def execute(self, userdata):
        global waypoints
        self.initialize_path_queue()
        self.path_ready = False

        # Start thread to listen for when the path is ready (this function will end then)
        def wait_for_path_ready():
            """thread worker function"""
            data = rospy.wait_for_message('/path_ready', Empty)
            rospy.loginfo('Recieved path READY message')
            self.path_ready = True
        ready_thread = threading.Thread(target=wait_for_path_ready)
        ready_thread.start()

        topic = self._custom_waypointstopic
        rospy.loginfo("Waiting to recieve waypoints via Pose msg on topic %s" % topic)
        rospy.loginfo("To start following waypoints: 'rostopic pub /path_ready std_msgs/Empty -1'")

        # Wait for published waypoints
        while not self.path_ready:
            try:
                pose = rospy.wait_for_message(topic, PoseWithCovarianceStamped, timeout=1)
            except rospy.ROSException as e:
                if 'timeout exceeded' in e.message:
                    continue  # no new waypoint within timeout, looping...
                else:
                    raise e
            rospy.loginfo("Recieved new waypoint")
            waypoints.append(pose)
            # publish waypoint queue as pose array so that you can see them in rviz, etc.
            self.poseArray_publisher.publish(convert_PoseWithCovArray_to_PoseArray(waypoints))

        # Path is ready! return success and move on to the next state (FOLLOW_PATH)
        return 'success'

class PathComplete(State):
    def __init__(self):
        State.__init__(self, outcomes=['success'])

    def execute(self, userdata):
        rospy.loginfo('###############################')
        rospy.loginfo('##### REACHED FINISH GATE #####')
        rospy.loginfo('###############################')
        return 'success'

def main():
    rospy.init_node('custom_follow_waypoints')

    sm = StateMachine(outcomes=['success'])

    with sm:
        StateMachine.add('GET_PATH', GetPath(),
                           transitions={'success':'FOLLOW_PATH'},
                           remapping={'waypoints':'waypoints'})
        StateMachine.add('FOLLOW_PATH', FollowPath(),
                           transitions={'success':'PATH_COMPLETE'},
                           remapping={'waypoints':'waypoints'})
        StateMachine.add('PATH_COMPLETE', PathComplete(),
                           transitions={'success':'GET_PATH'})

    outcome = sm.execute()

if __name__ == "__main__":
    main()

The new topic is retrived through a parameter setting.

In [ ]:
self._custom_waypointstopic = rospy.get_param('~custom_waypointstopic','/my_waypoints_list')

This means that when this node is launched, it will have to be set this parameter if we don't want it to have its default value **/my_waypoints_list**.<br>
Here you have an example of how it should be launched:

**start_follow_waypoints.launch**

In [ ]:
<launch>
    <env name="ROSCONSOLE_FORMAT" value="[${severity}][${thread}][${node}/${function}:${line}]: ${message}"/>
     <arg name="waypoints_topic" default="/my_sumit_xl_waypoints"/>
    
    <node pkg="my_sumit_xl_tools" type="custom_follow_waypoints.py" name="custom_follow_waypoints" output="screen" clear_params="true">
        <param name="goal_frame_id" value="map"/>
        <param name="custom_waypointstopic" value="$(arg waypoints_topic)"/>
    </node>

</launch>

Lets comment on various elements here:

In [ ]:
<env name="ROSCONSOLE_FORMAT" value="[${severity}][${thread}][${node}/${function}:${line}]: ${message}"/>

This line changes the way the logs are shown, in this case it will show which node, function and line in the code is that log executed.

In [ ]:
<arg name="waypoints_topic" default="/my_sumit_xl_waypoints"/>

Here you state which topic where you will have to publish the waypoint.

In [ ]:
<param name="goal_frame_id" value="map"/>
<param name="custom_waypointstopic" value="$(arg waypoints_topic)"/>

Here you state which is the frame and the topic params values.

<p style="background:#EE9023;color:white;">Exercise U1-3</p>

Now create a python script that generates those waypoints and when finished executes the moving system to follow those waypoints.<br>
To do so, you will need to publish directly into the topic **/my_sumit_xl_waypoints** or the value that you stated in tha launch file **start_follow_waypoints.launch**.<br>
Then you will have to publish to start in the topic **/path_ready**.<br>
Tasks:
* Make it generate circular paths
* Eight shaped paths
* Dynamic waypoints based on the position of the standing person. For this last one you will need to acces the TF data of the standingPerson.

<p style="background:#EE9023;color:white;">Exercise U1-3</p>

<p style="background:green;color:white;">Solution Exercise U.3</p>

Please Try to do it by yourself unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

Here you have an example of a publisher that generates a straight line waypoint path and executes it.

**waypoint_publisher.py**

In [ ]:
#!/usr/bin/env python
# license removed for brevity
import rospy
from std_msgs.msg import String
from geometry_msgs.msg import PoseWithCovarianceStamped
import time
import tf
from std_msgs.msg import Empty

"""
user:~$ rosmsg show geometry_msgs/PoseWithCovarianceStamped                                                                                                               
std_msgs/Header header                                                                                                                                                    
  uint32 seq                                                                                                                                                              
  time stamp                                                                                                                                                              
  string frame_id                                                                                                                                                         
geometry_msgs/PoseWithCovariance pose                                                                                                                                     
  geometry_msgs/Pose pose                                                                                                                                                 
    geometry_msgs/Point position                                                                                                                                          
      float64 x                                                                                                                                                           
      float64 y                                                                                                                                                           
      float64 z                                                                                                                                                           
    geometry_msgs/Quaternion orientation                                                                                                                                  
      float64 x                                                                                                                                                           
      float64 y                                                                                                                                                           
      float64 z                                                                                                                                                           
      float64 w                                                                                                                                                           
  float64[36] covariance 
"""

def talker():
    pub_wp = rospy.Publisher('/my_sumit_xl_waypoints', PoseWithCovarianceStamped, queue_size=1)
    
    pub_init_wp = rospy.Publisher('/path_ready', Empty, queue_size=1)
    
    rospy.init_node('waypoint_publisher', anonymous=True)
    rate = rospy.Rate(10) # 10hz
    
    my_wp = PoseWithCovarianceStamped()
    my_wp.header.stamp = rospy.Time.now()
    my_wp.header.frame_id = "/map"
    
    
    init_value = 2.0
    
    roll = 0
    pitch = 0
    yaw = 0.7
    quaternion = tf.transformations.quaternion_from_euler(roll, pitch, yaw)
    #type(pose) = geometry_msgs.msg.Pose
    my_wp.pose.pose.orientation.x = quaternion[0]
    my_wp.pose.pose.orientation.y = quaternion[1]
    my_wp.pose.pose.orientation.z = quaternion[2]
    my_wp.pose.pose.orientation.w = quaternion[3]


    for i in range(5):
        rospy.loginfo(str(i))
        my_wp.pose.pose.position.x = float(i) + init_value 
        my_wp.pose.pose.position.y = float(i) + init_value
        
        while not rospy.is_shutdown():
            connections = pub_wp.get_num_connections()
            if connections > 0:
                pub_wp.publish(my_wp)
                break
            rospy.loginfo("Waiting for /my_sumit_xl_waypoints topic")
            rate.sleep()
        rospy.loginfo("Published waypoint number ="+str(i))
        time.sleep(2)
    
    start_command = Empty()
    
    while not rospy.is_shutdown():
            connections = pub_init_wp.get_num_connections()
            if connections > 0:
                pub_init_wp.publish(start_command)
                rospy.loginfo("Sent waypoint list execution command")
                break
            rospy.loginfo("Waiting for /path_ready topic")
            rate.sleep()


if __name__ == '__main__':
    try:
        talker()
    except rospy.ROSInterruptException:
        pass

### Extra: Launch your RVIZ configs easily

Here you have an example of how to launch RVIZ sessions faster. You just have to create this tinny launch that retrieves the name of the **rviz** configuration file saved in the **rviz** folder. You will use it in the next units for sure.

**view_robot.launch**

In [ ]:
<launch>
  <param name="use_gui" value="true"/>
  <arg name="config" default="complete" />
  <node name="rviz" pkg="rviz" type="rviz" args="-d $(find my_summitxl_tools)/rviz/$(arg config).rviz" />
</launch>

And you can use it like so in any launch file:

In [ ]:
<!-- Start RVIZ for Localization -->
<include file="$(find my_sumit_xl_tools)/launch/view_robot.launch">
    <arg name="config" value="navigation_map" />
</include>

This one will load in RVIZ the file called **localization.rviz**.

### Congratulations! You now have a fully functional indoor navigating Summit XL. In next Chapter you will learn how to navigate outdoors using GPS data.